In [1]:
import pandas as pd
tripfile =r'C:\Users\hyunj\machin_prj\지역\수도권_지역.xlsx'
trip_Df = pd.read_excel(tripfile)

trip_Df.head(2)

,addr1,addr2,areacode,booktour,cat1,cat2,cat3,contentid,contenttypeid,createdtime,...,firstimage2,cpyrhtDivCd,mapx,mapy,mlevel,modifiedtime,sigungucode,tel,title,zipcode
0,서울특별시 송파구 새말로 114 (문정동),1층,1,NaN,음식,음식점,한식,3485244,39,20250400000000,...,http://tong.visitkorea.or.kr/cms/resource/39/3...,Type3,127.126643,37.482681,6.0,20250400000000,18.0,NaN,푼주,5810
1,서울특별시 성동구 무학로6길 22 (도선동),1층,1,NaN,음식,음식점,카페/전통찻집,3484142,39,20250300000000,...,http://tong.visitkorea.or.kr/cms/resource/41/3...,Type3,127.032527,37.565075,6.0,20250300000000,16.0,NaN,레이지 유스,4707


In [2]:
trip_Df = trip_Df[['addr1', 'cat1', 'cat2', 'cat3', 'mapx', 'mapy']]
trip_Df.head(2)

,addr1,cat1,cat2,cat3,mapx,mapy
0,서울특별시 송파구 새말로 114 (문정동),음식,음식점,한식,127.126643,37.482681
1,서울특별시 성동구 무학로6길 22 (도선동),음식,음식점,카페/전통찻집,127.032527,37.565075


In [3]:
subway_file =r'C:\Users\hyunj\machin_prj\대중교통위치\전체역_업데이트.xlsx'
subway_Df = pd.read_excel(subway_file)
subway_Df.head(2)

,line,name,code,위도,경도
0,01호선,녹양역,1908.0,37.759380,127.042292
1,01호선,남영역,1002.0,37.541021,126.971300


In [4]:
# 그룹별 순서(order) 컬럼 추가
subway_Df['order'] = subway_Df.groupby('name').cumcount() + 1

# 피벗 수행: station별 line을 line1, line2로 분리
df_pivot = subway_Df.pivot(index='name', columns='order', values='line')
df_pivot.columns = [f'line{col}' for col in df_pivot.columns]
df_pivot = df_pivot.reset_index()
df_pivot


,name,line1,line2,line3,line4
0,419민주묘지역,우이신설경전철,NaN,NaN,NaN
1,가능역,01호선,NaN,NaN,NaN
2,가락시장역,03호선,08호선,NaN,NaN
3,가산디지털단지역,01호선,07호선,NaN,NaN
4,가양역,09호선,NaN,NaN,NaN
...,...,...,...,...,...
611,회현역,04호선,NaN,NaN,NaN
612,효자역,의정부경전철,NaN,NaN,NaN
613,효창공원앞역,06호선,경의선,NaN,NaN
614,흑석역,09호선,NaN,NaN,NaN


In [5]:
bus_file = r'C:\Users\hyunj\machin_prj\대중교통위치\국토교통부_전국 버스정류장 위치정보_20241028.csv'

bus_Df = pd.read_csv(bus_file, encoding='cp949')
bus_Df = bus_Df[bus_Df['도시명'].str.contains(r'^(서울특별시|경기도|인천광역시)', na=False, regex=True)]

bus_DF =bus_Df[["정류장명","위도","경도","도시명"]]
bus_DF.head(2)

C:\Users\hyunj\AppData\Local\Temp\ipykernel_23232\2857500830.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  bus_Df = bus_Df[bus_Df['도시명'].str.contains(r'^(서울특별시|경기도|인천광역시)', na=False, regex=True)]


,정류장명,위도,경도,도시명
53999,도봉구민회관.도봉문화원(중),37.654450,127.038067,서울특별시
54000,도봉구민회관.도봉문화원(중),37.654117,127.038033,서울특별시


In [6]:
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # 지구 반경 (km)
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)

    a = np.sin(dlat / 2)**2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    return R * c  # 거리 (km)

# 가장 가까운 지하철역 또는 버스 정류장 찾기
def find_nearest_station(lat, lon, subway_Df, bus_DF):
    nearest_subway = None
    nearest_bus = None
    min_subway_dist = float('inf')
    min_bus_dist = float('inf')

    # 지하철역 거리 계산
    for _, row in subway_Df.iterrows():
        distance = haversine(lat, lon, row['위도'], row['경도'])
        if distance < min_subway_dist:
            min_subway_dist = distance
            nearest_subway = row['line'] + " " + row['name']

    # 버스 정류장 거리 계산
    for _, row in bus_DF.iterrows():
        distance = haversine(lat, lon, row['위도'], row['경도'])
        if distance < min_bus_dist:
            min_bus_dist = distance
            nearest_bus = row['정류장명']

    # 둘 중 더 가까운 곳 선택
    return nearest_subway if min_subway_dist < min_bus_dist else nearest_bus

# 새로운 열 추가
trip_Df['nearest_station'] = trip_Df.apply(lambda row: find_nearest_station(row['mapy'], row['mapx'], subway_Df, bus_DF), axis=1)



In [7]:
trip_Df.to_excel('my_dataframe.xlsx', index=False)  
trip_Df

,addr1,cat1,cat2,cat3,mapx,mapy,nearest_station
0,서울특별시 송파구 새말로 114 (문정동),음식,음식점,한식,127.126643,37.482681,문정법조단지.건영아파트(중)
1,서울특별시 성동구 무학로6길 22 (도선동),음식,음식점,카페/전통찻집,127.032527,37.565075,상왕십리역
2,서울특별시 강서구 마곡중앙로 161-8 (마곡동),음식,음식점,이색음식점,126.827126,37.568982,마곡나루역1번출구
3,서울특별시 성북구 동소문로20다길 30 (동선동1가),음식,음식점,카페/전통찻집,127.018306,37.592083,돈암사거리.성신여대입구(중)
4,서울특별시 강북구 도봉로68길 36 (미아동),음식,음식점,카페/전통찻집,127.026445,37.631395,수유시장.성신여대미아캠퍼스앞(중)
...,...,...,...,...,...,...,...
14128,경기도 화성시 효행로1265번길 76,음식,음식점,카페/전통찻집,127.055138,37.225603,반월중.고등학교앞
14129,경기도 화성시 효행로1374번길 24,음식,음식점,카페/전통찻집,127.067525,37.229089,신동탄롯데캐슬
14130,경기도 화성시 효행로481번길 21 (안녕동),인문(문화/예술/역사),역사관광지,유적지/사적지,126.989286,37.208185,융건릉입구
14131,경기도 화성시 효행로481번길 21 (안녕동),인문(문화/예술/역사),역사관광지,유적지/사적지,126.989286,37.208185,융건릉입구


In [8]:
# 거리 계산 함수
def get_closest_station_and_bus(row, subway_Df, bus_DF):
    # subway와 bus 정류장들에 대한 위도/경도 값
    subway_coords = subway_Df[['위도', '경도']].values
    bus_coords = bus_DF[['위도', '경도']].values

    # trip의 현재 좌표
    trip_coord = (row['mapy'], row['mapx'])

    # subway에서 가장 가까운 역 찾기
    subway_distances = [haversine(trip_coord[0], trip_coord[1], coord[0], coord[1]) for coord in subway_coords]
    closest_subway_index = np.argmin(subway_distances)
    closest_subway = subway_coords[closest_subway_index]

    # bus에서 가장 가까운 정류장 찾기
    bus_distances = [haversine(trip_coord[0], trip_coord[1], coord[0], coord[1]) for coord in bus_coords]
    closest_bus_index = np.argmin(bus_distances)
    closest_bus = bus_coords[closest_bus_index]

    # 가장 가까운 지하철역
    closest_subway_station = subway_Df.iloc[closest_subway_index]['name']
    closest_subway_line = subway_Df.iloc[closest_subway_index]['line']

    # 가장 가까운 버스정류장
    closest_bus_station = bus_DF.iloc[closest_bus_index]['정류장명']

    return closest_subway_station, closest_subway_line, closest_bus_station

# trip_Df에 가장 가까운 지하철역과 버스정류장 추가
trip_Df[['closest_subway_station', 'closest_subway_line', 'closest_bus_station']] = trip_Df.apply(
    lambda row: pd.Series(get_closest_station_and_bus(row, subway_Df, bus_DF)), axis=1
)

#

In [9]:
trip_Df['closest_bus_station'] = trip_Df['closest_bus_station'].str.replace(r'\(.*\)$', '', regex=True)

trip_Df.to_excel('my_dataframe1.xlsx', index=False)  
trip_Df.head(2)

,addr1,cat1,cat2,cat3,mapx,mapy,nearest_station,closest_subway_station,closest_subway_line,closest_bus_station
0,서울특별시 송파구 새말로 114 (문정동),음식,음식점,한식,127.126643,37.482681,문정법조단지.건영아파트(중),장지역,08호선,문정법조단지.건영아파트
1,서울특별시 성동구 무학로6길 22 (도선동),음식,음식점,카페/전통찻집,127.032527,37.565075,상왕십리역,상왕십리역,02호선,상왕십리역


In [10]:
station_choices = bus_DF['정류장명'].unique()
station_choices

array(['도봉구민회관.도봉문화원(중)', '쌍문역(중)', '우이1교앞(중)', ..., '540번종점', '대원정밀',
       '부평농장사무소'], dtype=object)

In [13]:
# closest_bus_station을 기준으로 train_Df에 bus_Df의 정류장번호 추가
result_df = pd.merge(trip_Df, bus_Df[['정류장명', '정류장번호']], how='left', left_on='closest_bus_station', right_on='정류장명')

# result_df = pd.merge(trip_Df, bus_Df[['정류장명', '정류장번호']], how='left', left_on='matched_station_name', right_on='정류장명')


# # 필요한 열만 추출하여 결과 생성
# result_df = result_df.drop(columns=['정류장명'])
# result_df = result_df.drop(columns=['closest_bus_station'])

result_df.head(2)

,addr1,cat1,cat2,cat3,mapx,mapy,nearest_station,closest_subway_station,closest_subway_line,closest_bus_station,정류장명,정류장번호
0,서울특별시 송파구 새말로 114 (문정동),음식,음식점,한식,127.126643,37.482681,문정법조단지.건영아파트(중),장지역,08호선,문정법조단지.건영아파트,NaN,NaN
1,서울특별시 성동구 무학로6길 22 (도선동),음식,음식점,카페/전통찻집,127.032527,37.565075,상왕십리역,상왕십리역,02호선,상왕십리역,상왕십리역,GGB103000005


In [3]:
# result_df.to_excel('my_dataframe3.xlsx', index=False)  
result_df.head(2)

NameError: name 'result_df' is not defined

In [ ]:
result_df.head(2)

In [15]:
import pandas as pd
tripfile =r"C:\Users\hyunj\machin_prj\지역\수도권_지역_emd추가.xlsx"
trip_Df2 = pd.read_excel(tripfile)

trip_Df2.head(2)

,addr1,addr2,areacode,booktour,cat1,cat2,cat3,contentid,contenttypeid,createdtime,...,cpyrhtDivCd,mapx,mapy,mlevel,modifiedtime,sigungucode,tel,title,zipcode,emd
0,서울특별시 송파구 새말로 114 (문정동),1층,1,NaN,음식,음식점,한식,3485244,39,20250400000000,...,Type3,127.126643,37.482681,6.0,20250400000000,18.0,NaN,푼주,5810,문정동
1,서울특별시 성동구 무학로6길 22 (도선동),1층,1,NaN,음식,음식점,카페/전통찻집,3484142,39,20250300000000,...,Type3,127.032527,37.565075,6.0,20250300000000,16.0,NaN,레이지 유스,4707,도선동


In [1]:
result_df.to_excel('my_dataframe4.xlsx', index=False)  
result_df.head(2)

NameError: name 'result_df' is not defined

In [16]:
result_df = pd.merge(result_df, df_pivot.rename(columns={'name': 'closest_subway_station'}), 
                     how='left', on='closest_subway_station')
# 'closest_subway_line' 열 삭제
result_df.drop(columns=['closest_subway_line'], inplace=True)
result_df.drop(columns=['line1'], inplace=True)
result_df.drop(columns=['line2'], inplace=True)
result_df.drop(columns=['line3'], inplace=True)
result_df.drop(columns=['line4'], inplace=True)
# 결과 확인
result_df

,addr1,cat1,cat2,cat3,mapx,mapy,nearest_station,closest_subway_station,closest_bus_station,정류장명,정류장번호
0,서울특별시 송파구 새말로 114 (문정동),음식,음식점,한식,127.126643,37.482681,문정법조단지.건영아파트(중),장지역,문정법조단지.건영아파트,NaN,NaN
1,서울특별시 성동구 무학로6길 22 (도선동),음식,음식점,카페/전통찻집,127.032527,37.565075,상왕십리역,상왕십리역,상왕십리역,상왕십리역,GGB103000005
2,서울특별시 성동구 무학로6길 22 (도선동),음식,음식점,카페/전통찻집,127.032527,37.565075,상왕십리역,상왕십리역,상왕십리역,상왕십리역,GGB103000015
3,서울특별시 강서구 마곡중앙로 161-8 (마곡동),음식,음식점,이색음식점,126.827126,37.568982,마곡나루역1번출구,마곡나루역,마곡나루역1번출구,마곡나루역1번출구,ICB115000898
4,서울특별시 성북구 동소문로20다길 30 (동선동1가),음식,음식점,카페/전통찻집,127.018306,37.592083,돈암사거리.성신여대입구(중),성신여대입구역,돈암사거리.성신여대입구,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
30167,경기도 화성시 효행로481번길 21 (안녕동),인문(문화/예술/역사),역사관광지,유적지/사적지,126.989286,37.208185,융건릉입구,병점역,융건릉입구,융건릉입구,GGB233001840
30168,경기도 화성시 효행로481번길 21 (안녕동),인문(문화/예술/역사),역사관광지,유적지/사적지,126.989286,37.208185,융건릉입구,병점역,융건릉입구,융건릉입구,GGB233001841
30169,경기도 화성시 효행로481번길 21 (안녕동),인문(문화/예술/역사),역사관광지,유적지/사적지,126.989286,37.208185,융건릉입구,병점역,융건릉입구,융건릉입구,GGB233001840
30170,경기도 화성시 효행로481번길 21 (안녕동),인문(문화/예술/역사),역사관광지,유적지/사적지,126.989286,37.208185,융건릉입구,병점역,융건릉입구,융건릉입구,GGB233001841


In [17]:
import pandas as pd

# `addr1`을 기준으로 `trip_Df2`와 `result_df` 병합
result_df = pd.merge(result_df, trip_Df2[['addr1', 'emd']], how='left', on='addr1')
result_df = pd.merge(result_df, trip_Df2[['addr1', 'title']], how='left', on='addr1')

result_df.head(2)


,addr1,cat1,cat2,cat3,mapx,mapy,nearest_station,closest_subway_station,closest_bus_station,정류장명,정류장번호,emd,title
0,서울특별시 송파구 새말로 114 (문정동),음식,음식점,한식,127.126643,37.482681,문정법조단지.건영아파트(중),장지역,문정법조단지.건영아파트,NaN,NaN,문정동,푼주
1,서울특별시 성동구 무학로6길 22 (도선동),음식,음식점,카페/전통찻집,127.032527,37.565075,상왕십리역,상왕십리역,상왕십리역,상왕십리역,GGB103000005,도선동,레이지 유스


In [ ]:
result_df.to_excel('my_dataframe5.xlsx', index=False)  
result_df.head(2)